---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kpcrotty.github.io/)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [61]:
# means
mn1 = 0.08
mn2 = 0.12
mn3 = 0.15

# std devs
sd1 = 0.15
sd2 = 0.25
sd3 = 0.35

# correlations
c12 = 0.15
c13 = 0.60
c23 = 0.30

# INSTALL LIBRARIES

In [62]:
!pip install cvxopt


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# CREATE ARRAYS

In [63]:
import numpy as np 

mn = np.array([mn1, mn2, mn3])
sd = np.array([sd1, sd2, sd3])

corr = np.identity(3)
corr[0, 1] = corr[1, 0] = c12
corr[0, 2] = corr[2, 0] = c13
corr[1, 2] = corr[2, 1] = c23

cov = np.diag(sd) @ corr @ np.diag(sd)

# CHECK CORRELATIONS

In [64]:
if np.all(np.linalg.eigvals(cov) > 0):
    print("correlations are acceptable")
else:
    print("correlations are inconsistent")

correlations are acceptable


# FRONTIER WITH SHORT SALES

In [65]:
# global minimum variance portfolio
w = np.linalg.solve(cov, np.ones(3))
gmv = w / np.sum(w)
gmv_mn = gmv @ mn
gmv_sd = np.sqrt(gmv @ cov @ gmv)

# second frontier portfolio
w = np.linalg.solve(cov, mn)
second_port = w / np.sum(w)
second_mn = second_port @ mn 
second_sd = np.sqrt(second_port @ cov @ second_port)

# means to display
min_mn = 0
max_mn = 1.2*np.max(mn)
mns = np.linspace(min_mn, max_mn, 101)

# portfolio weights and risks
gmv_wt = (mns - second_mn) / (gmv_mn - second_mn)
second_wt = 1 - gmv_wt
ports_shorts = (
    gmv_wt.reshape(-1, 1) * gmv.reshape(1, -1) + 
    second_wt.reshape(-1, 1) * second_port.reshape(1, -1)
)
vr = np.diag(ports_shorts @ cov @ ports_shorts.T)
sds_shorts = np.sqrt(vr)


# FRONTIER WITHOUT SHORT SALES

For a given target expected return, we want to find the minimum risk portfolio that has only nonnegative positions.  This means we want to solve

    min w @ cov @ w subject to w >= 0, w @ np.ones = 1, and w @ mn = target

In [66]:
from cvxopt.solvers import options
options["show_progress"] = False

# minimum risk to achieve a target mean return
def constrained(target):
    
    # to define variance
    P = cov

    # no linear term in objective
    q = np.zeros(3)

    # to impose -w <= 0
    G = -np.identity(3)
    h = np.zeros(3)

    # to impose w1 + w2 + w3 = 1 and w @ mn = target
    A = np.vstack(([1., 1., 1.], mn))
    b = np.vstack(([1., target]))
  
    # create cvxopt matrix objects
    P = matrix(P, (3, 3))
    q = matrix(q, (3, 1))
    G = matrix(G, (3, 3))
    h = matrix(h, (3, 1))
    A = matrix(A, (2, 3))
    b = matrix(b, (2, 1))

    # compute optimum by quadratic programming
    solution = qp(P=P, q=q, G=G, h=h, A=A, b=b)
    return np.array(solution["x"]).reshape(1, -1)

targets = np.linspace(np.min(mn), np.max(mn), 50)
ports_constrained = [constrained(t) for t in targets]
ports_constrained = np.vstack(ports_constrained)
sds_constrained = np.sqrt(np.diag(ports_constrained @ cov @ ports_constrained.T))


# FIGURE

In [67]:
import plotly.graph_objects as go 

# frontier with short sales
string = 'asset 1: %{customdata[0]:.0%}<br>'
string += 'asset 2: %{customdata[1]:.0%}<br>'
string += 'asset 3: %{customdata[2]:.0%}<br>'
string += '<extra></extra>'
tr1 = go.Scatter(
    x=sds_shorts,
    y=mns,
    mode="lines",
    customdata=ports_shorts,
    hovertemplate=string,
    name="frontier with short sales"
)

# frontier without short sales
tr2 = go.Scatter(
    x=sds_constrained,
    y=targets,
    mode="lines",
    customdata=ports_constrained,
    hovertemplate=string,
    name="frontier without short sales"
)

fig = go.Figure(tr1)
fig.add_trace(tr2)
fig.update_layout(
    xaxis_title="Standard Deviation",
    yaxis_title="Expected Return",
    xaxis_tickformat=".0%",
    yaxis_tickformat=".0%",
    xaxis_rangemode="tozero",
    yaxis_rangemode="tozero",
    template="plotly_white",
    legend=dict(
        xanchor="left",
        yanchor="top",
        y=0.99,
        x=0.01,
    )
) 
fig.show()
